<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/TF_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создаем простой граф


In [0]:
import tensorflow as tf

x = tf.Variable (3, name="x")
y = tf.Variable (4, name="y")
f = x*x*y + y + 2


**Добавляем сессию**

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


**Упращенная сессия (с автозакрытием)**

In [3]:
with tf.Session() as sess:
  x.initializer.run()
  y.initializer.run()
  result = f.eval()
  print(result)

42


**Глобальная инициализация**

In [4]:
import tensorflow as tf

x = tf.Variable(3, "x")
y = tf.Variable(2, "y")
z = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
  init.run()
  result = f.eval()
  print(result)

42


**Новый отдельный экземпляр графа **

In [5]:
graph = tf.Graph()
with graph.as_default():
  x2 = tf.Variable(2)
  y2 = tf.Variable(6)
  Z2 = x2*y2
  
  init = tf.global_variables_initializer()
  sess = tf.InteractiveSession()
  init.run()
  result2 = Z2.eval()
  print(result2)
  sess.close()
  

12


**Оценка в одном прогоне**

In [6]:
import tensorflow as tf
w = tf.constant(3)
x = w + 2
y = x + 5 
z = x * 3

init = tf.global_variables_initializer()
with tf.Session() as sess:
  init.run()
  y_val, z_val = sess.run([y,z])
  print(y_val)
  print(z_val)
 
  sess.close()


10
15


**Линейная регресия упрощенная**

In [7]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()   
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
m, n = housing.data.shape

#print(housing.feature_names)
#print(housing.DESCR)
#print(housing.target.shape)

# np.ones создает новый array заполненный 1ми, np.c_ создает новый масив обьединяя 2
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant (housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant (housing.target.reshape(-1,1), dtype=tf.float32, name="y")

XT = tf.transpose(X)
theta = tf.matmul (tf.matmul (tf.matrix_inverse(tf.matmul(XT,X)), XT) , y)

with tf.Session() as sess:
  theta_value = theta.eval ()
  print(theta_value)

NameError: ignored

**Вычисление градиента в ручную и автоматически**

In [28]:




import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

m, n = housing.data.shape

n_epochs = 10
learning_rate = 0.001

# нормализация данных
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

# np.c_ сращивание матриц, np.ones создание матрицы заполненой нулями
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

# tf.random_uniform генерировать случайный тензер с заданой формой и диапазоном
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta") 
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y

# tf.reduce_mean вычисляет среднее значение
mse = tf.reduce_mean(tf.square(error), name="mse")

#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients (mse, [theta])[0]

# создает узел который будет присваивать переменной новое значение 
training_op = tf.assign(theta, theta - learning_rate * gradients) 
init = tf.global_variables_initializer()
 

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 2 == 0:
           k = y_pred.eval()
           print("Epoch", epoch, "MSE =", mse.eval(), "y_pred", y_pred.shape, "theta", theta.shape, "error", error.shape, "X", X.shape)
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.7544262 y_pred (20640, 1) theta (9, 1) error (20640, 1) X (20640, 9)
Epoch 2 MSE = 2.7365942 y_pred (20640, 1) theta (9, 1) error (20640, 1) X (20640, 9)
Epoch 4 MSE = 2.7189174 y_pred (20640, 1) theta (9, 1) error (20640, 1) X (20640, 9)
Epoch 6 MSE = 2.7013936 y_pred (20640, 1) theta (9, 1) error (20640, 1) X (20640, 9)
Epoch 8 MSE = 2.684023 y_pred (20640, 1) theta (9, 1) error (20640, 1) X (20640, 9)


 **Вывод матрицы/тензора **

In [20]:
import tensorflow as tf
g = tf.random_uniform([10 + 1, 1], 0.0, 10.0, seed=42)
init_op = tf.initialize_all_variables()
h = tf.transpose(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42))
k = tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42)
with tf.Session() as sess:
    sess.run(init_op) 
    print (sess.run(h))#распечатать тензор
    print (sess.run(k))#распечатать тензор
    print(g.get_shape) #получить размер массива статич

       




[[ 0.9045429   0.35481548  0.5906365   0.51156354 -0.04808879  0.26202965
  -0.62795925 -0.7713845  -0.32755637]]
[[ 0.9045429 ]
 [ 0.35481548]
 [ 0.5906365 ]
 [ 0.51156354]
 [-0.04808879]
 [ 0.26202965]
 [-0.62795925]
 [-0.7713845 ]
 [-0.32755637]]
<bound method Tensor.get_shape of <tf.Tensor 'random_uniform_22:0' shape=(11, 1) dtype=float32>>


**Использование оптимизатора**

In [0]:
import tensorflow as tf
import numpy as np


from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler


housing = fetch_california_housing()
m, n = housing.data.shape

learning_rate = 0.01
n_epochs = 4

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)

scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

def fetch_batch (batch_index, batch_size, epoch):
    np.random.seed (epoch * batch_size + batch_size) 
    indices = np.random.randint (m, size=batch_size) 
    X_batch = scaled_housing_data_plus_bias [indices]
    y_batch = housing.target.reshape(-1, 1) [indices]
    return X_batch, y_batch




In [34]:
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="prediction")
erorr = y_pred - y
mse = tf.reduce_mean(tf.square(erorr), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), erorr, name="gradients")
#training_op = tf.assign(theta, theta - learning_rate * gradients)

optimizer = tf.train.GradientDescentOptimizer (learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

batch_size = 3
n_batches = int(np.ceil(6 / batch_size))

init = tf.global_variables_initializer()
with tf.Session() as sess:
     sess.run(init)
     for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            #print("X_batch", X_batch, "Theta", theta.eval())
            print("Epoch", epoch, "X", X.shape)
        #loss_val = loss.eval({X: X_test, y: y_test})
        #if epoch % 2 == 0 :
            #print("Epoch:", epoch, "MSE:")



ValueError: ignored

In [7]:
print(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42))

Tensor("random_uniform_5:0", shape=(9, 1), dtype=float32)
